In [1]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyMuPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

c:\Users\Admin\.conda\envs\mchatbot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
PINECONE_API_KEY="68f74fa6-7dd3-49d7-acd7-964ff69ae43f"

In [3]:
def load_pdf(pdf_file_path):
    loader = PyMuPDFLoader(pdf_file_path)
    document = loader.load()
    return document


In [4]:
extracted_data=load_pdf("D:\Lenditt\MD_House\Medical-Chatbot-using-llama2\Medical_book.pdf")

In [5]:
# Creaate text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [6]:
text_chunks = text_split(extracted_data)
print("length of text_chunks", len(text_chunks))

length of text_chunks 5779


In [7]:
# download embedding model
def download_hugging_face_embedding():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [8]:
embeddings = download_hugging_face_embedding()

In [9]:
querry_result = embeddings.embed_query("Hello World")
print("Length", len(querry_result))

Length 384


In [10]:
from pinecone import Pinecone
from transformers import AutoTokenizer, AutoModel
import torch
import numpy


# Initialize Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)

# Create or retrieve an index
index = pc.Index("medicalchatbot")

# Assuming you have a pre-trained model and text data
model_name = "sentence-transformers/all-MiniLM-L6-v2"  # Replace with your desired model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Process your text chunks
text_data = [t.page_content for t in text_chunks]
embeddings = []
for text in text_data:
  # Tokenize and get embeddings
  input_ids = tokenizer(text, return_tensors="pt")
  with torch.no_grad():
    output = model(**input_ids)
  embeddings.append(output.last_hidden_state.detach().numpy())
  
# Convert embeddings to a list of flattened arrays (assuming multiple dimensions)
float_embeddings = [embedding.flatten().tolist() for embedding in embeddings]

# Index your text chunks
index.upsert(texts=text_data, vectors=float_embeddings)



ValueError: Invalid vector value passed: cannot interpret type <class 'list'>